# Project Introduction: Integrating MA Crossover and MACD for Stock Analysis

## Objective
The objective of this project is to develop a reliable stock screening tool that identifies potential buy and sell signals using two popular technical indicators: the Moving Average (MA) Crossover and the Moving Average Convergence Divergence (MACD). By combining these indicators, we aim to enhance the accuracy of our signals, leveraging the strengths of both trend-following and momentum analysis.

## Background
Technical analysis involves using historical price and volume data to forecast future price movements. Among the myriad of technical indicators available, MA Crossover and MACD are two widely respected tools that traders use to assess market conditions.

## Why MA Crossover and MACD?

### Moving Average (MA) Crossover
- **Purpose:** The MA Crossover is used to identify potential trend reversals by examining the crossing points of different moving averages.
- **Components:**
  - **Short-Term MA:** A moving average calculated over a shorter period (e.g., 20-day MA).
  - **Long-Term MA:** A moving average calculated over a longer period (e.g., 100-day MA).
- **Potential Trend Reversals:**
  - **Potential Bullish Trend Reversal:** Occurs when the short-term MA crosses above the long-term MA, indicating a potential buy signal.
  - **Potential Bearish Trend Reversal:** Occurs when the short-term MA crosses below the long-term MA, indicating a potential sell signal.

### Moving Average Convergence Divergence (MACD)
- **Purpose:** The MACD is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price.
- **Components:**
  - **MACD Line:** The difference between the 12-day EMA and the 26-day EMA.
  - **Signal Line:** The 9-day EMA of the MACD line.
  - **Histogram:** The difference between the MACD line and the Signal line.
- **Potential Trend Reversals:**
  - **Potential Bullish Trend Reversal:** When the MACD line crosses above the Signal line, indicating upward momentum.
  - **Potential Bearish Trend Reversal:** When the MACD line crosses below the Signal line, indicating downward momentum.

## Combining MA Crossover and MACD
Using these indicators together can help confirm signals and reduce false positives:
- **Enhanced Accuracy:** While the MA Crossover helps identify potential trend changes, MACD provides context by showing the momentum of the price movement during these changes.
- **Filter for False Signals:** The MA Crossover can sometimes give premature signals. By requiring confirmation through MACD's momentum context, we filter out some of these false signals, increasing the likelihood that our signals are accurate.

## Methodology
1. **Data Collection:** We collect historical price data for a selection of stocks over a specified period.
2. **Indicator Calculation:** We calculate the MA Crossover and MACD for each stock.
3. **Signal Generation:** We generate potential buy and sell signals based on the following criteria:
   - **Potential Bullish Trend Reversal:** Short-term MA crosses above the long-term MA and MACD line crosses above the Signal line.
   - **Potential Bearish Trend Reversal:** Short-term MA crosses below the long-term MA and MACD line crosses below the Signal line.
4. **Visualization and Export:** We visualize the results and export the signals to a CSV file for further analysis.

## Data Source and Flexibility
In this analysis, we will be using data from the FTSE 250 companies. However, feel free to use any dataset you prefer by simply changing the CSV file name in the script. Ensure that your dataset contains a column named "Ticker".

## Conclusion
This project aims to leverage the complementary strengths of MA Crossover and MACD to create a robust stock screening tool. By considering both trend-following and momentum, we can provide more reliable buy and sell signals, helping traders make more informed decisions. The chosen thresholds and periods are standard and widely accepted in technical analysis, ensuring that our signals are both practical and effective.


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
import contextlib
import sys
import os

# Function to suppress print statements
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# Function to calculate Moving Averages and detect crossovers
def calculate_moving_averages(data, short_window=15, long_window=70):
    data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window).mean()
    data['Signal'] = np.where(data['Short_MA'] > data['Long_MA'], 1, 0)
    data['Crossover'] = data['Signal'].diff()
    return data

# Function to calculate MACD
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    data['EMA12'] = data['Close'].ewm(span=short_window, adjust=False).mean()
    data['EMA26'] = data['Close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA12'] - data['EMA26']
    data['MACD_Signal'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    data['MACD_Hist'] = data['MACD'] - data['MACD_Signal']
    return data

# Function to analyze stock for MA crossovers and MACD signals
def analyze_stock_ma_macd(symbol):
    try:
        end_date = datetime.now() - timedelta(days=1)
        start_date = end_date - timedelta(days=9*30)
        
        with suppress_output():
            data = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        
        if data.empty:
            return {"Symbol": symbol, "Data": pd.DataFrame(), "Meets Criteria": False}

        # Calculate MA Crossover and MACD
        data = calculate_moving_averages(data)
        data = calculate_macd(data)
        data.dropna(subset=['Short_MA', 'Long_MA', 'Crossover', 'MACD', 'MACD_Signal'], inplace=True)

        if data.empty:
            return {"Symbol": symbol, "Data": pd.DataFrame(), "Meets Criteria": False}

        # Determine MA crossover conditions
        recent_crossover = data['Crossover'].iloc[-5:].values
        bullish_trend_change = 1 in recent_crossover
        bearish_trend_change = -1 in recent_crossover

        # Ensure mutual exclusivity for trend changes
        if bullish_trend_change:
            bearish_trend_change = False
        elif bearish_trend_change:
            bullish_trend_change = False

        # Check the last MACD and Signal values
        macd_crossover = data['MACD'].iloc[-1] > data['MACD_Signal'].iloc[-1]
        macd_crossunder = data['MACD'].iloc[-1] < data['MACD_Signal'].iloc[-1]

        # Ensure mutual exclusivity for MACD signals
        if macd_crossover:
            macd_crossunder = False
        elif macd_crossunder:
            macd_crossover = False

        # Determine if the stock meets both criteria
        meets_criteria = (bullish_trend_change and macd_crossover) or (bearish_trend_change and macd_crossunder)

        return {
            "Symbol": symbol,
            "Short_MA": data['Short_MA'].iloc[-1],
            "Long_MA": data['Long_MA'].iloc[-1],
            "MACD": data['MACD'].iloc[-1],
            "MACD_Signal": data['MACD_Signal'].iloc[-1],
            "Bullish Trend Change": bullish_trend_change,
            "Bearish Trend Change": bearish_trend_change,
            "MACD_Crossover": macd_crossover,
            "MACD_Crossunder": macd_crossunder,
            "Meets Criteria": meets_criteria,
            "Data": data
        }

    except Exception as e:
        print(f"Error analyzing stock {symbol}: {e}")
        return {"Symbol": symbol, "Data": pd.DataFrame(), "Meets Criteria": False}


# Load tickers from the CSV file
tickers_df = pd.read_csv('ftse_250_tickers.csv')
symbols = tickers_df['Ticker'].tolist()

# Initialize list to store results
results = []

# Analyze each stock
for symbol in tqdm(symbols, desc="Processing stocks"):
    stock_data = analyze_stock_ma_macd(symbol)
    if stock_data is not None and stock_data["Meets Criteria"]:
        results.append(stock_data)

# Convert results to DataFrame
results_df = pd.DataFrame([{
    "Symbol": r["Symbol"],
    "Short_MA": r["Short_MA"],
    "Long_MA": r["Long_MA"],
    "MACD": r["MACD"],
    "MACD_Signal": r["MACD_Signal"],
    "Bullish Trend Change": r["Bullish Trend Change"],
    "Bearish Trend Change": r["Bearish Trend Change"],
    "MACD_Crossover": r["MACD_Crossover"],
    "MACD_Crossunder": r["MACD_Crossunder"],
    "Meets Criteria": r["Meets Criteria"]
} for r in results])

# Export results to CSV
results_df.to_csv("ma_cross_macd_screener_results.csv", index=False)


# Print results
results_df



In [ ]:
# Calculate and print the detailed results
total_tickers = len(tickers_df)
tickers_meeting_criteria = len(results_df)
percentage_meeting_criteria = (tickers_meeting_criteria / total_tickers) * 100

# Print results
print("My preference is a 5-20% threshold which strikes a balance, ensuring a\nfocused yet diverse selection of stocks to allow further investigations.")
print(f"Tickers meeting criteria: {tickers_meeting_criteria}")
print(f"Percentage of shares that pass the criteria: {percentage_meeting_criteria:.2f}%")

# Determine if the percentage is within the desired filter range
if 5 <= percentage_meeting_criteria <= 20:
    print("The percentage of shares meeting the criteria is within the desirable range (5-20%).")
else:
    print("The percentage of shares meeting the criteria is outside the desirable range (5-15%).")

In [ ]:
# Function to print basic financial data
def print_financial_data(ticker, stock_data):
    stock = yf.Ticker(ticker)
    info = stock.info

    def format_value(value, currency=False):
        if isinstance(value, (int, float)):
            formatted_value = f"{value:,}"
            if currency:
                formatted_value = f"£{formatted_value}"
            return formatted_value
        return value

    market_cap = format_value(info.get('marketCap', 'N/A'), currency=True)
    shares_outstanding = format_value(info.get('sharesOutstanding', 'N/A'))
    year_high = info.get('fiftyTwoWeekHigh', 'N/A')
    year_low = info.get('fiftyTwoWeekLow', 'N/A')
    pe_ratio = info.get('trailingPE', 'N/A')
    eps = info.get('trailingEps', 'N/A')
    dividend_yield = info.get('dividendYield', 'N/A')
    payout_ratio = info.get('payoutRatio', 'N/A')
    revenue = format_value(info.get('totalRevenue', 'N/A'), currency=True)
    gross_profit = format_value(info.get('grossProfits', 'N/A'), currency=True)
    net_income = format_value(info.get('netIncomeToCommon', 'N/A'), currency=True)
    total_debt = format_value(info.get('totalDebt', 'N/A'), currency=True)
    operating_cash_flow = format_value(info.get('operatingCashflow', 'N/A'), currency=True)
    free_cash_flow = format_value(info.get('freeCashflow', 'N/A'), currency=True)
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')
    description = info.get('longBusinessSummary', 'N/A')

    print(f"{'Ticker:':<25} {ticker}")
    print(f"{'Company:':<25} {info.get('shortName', 'N/A')}")
    print(f"{'Sector:':<25} {sector}")
    print(f"{'Industry:':<25} {industry}")
    print(f"{'Market Cap:':<25} {market_cap}")
    print(f"{'Shares Outstanding:':<25} {shares_outstanding}")
    print(f"{'52-Week High:':<25} {year_high}")
    print(f"{'52-Week Low:':<25} {year_low}")
    print(f"{'PE Ratio (TTM):':<25} {pe_ratio}")
    print(f"{'EPS (TTM):':<25} {eps}")
    print(f"{'Dividend Yield:':<25} {dividend_yield}")
    print(f"{'Payout Ratio:':<25} {payout_ratio}")
    print(f"{'Revenue (TTM):':<25} {revenue}")
    print(f"{'Gross Profit (TTM):':<25} {gross_profit}")
    print(f"{'Net Income (TTM):':<25} {net_income}")
    print(f"{'Total Debt:':<25} {total_debt}")
    print(f"{'Operating Cash Flow (TTM):':<25} {operating_cash_flow}")
    print(f"{'Free Cash Flow (TTM):':<25} {free_cash_flow}")
    print(f"\n{'Description:':<25} {description}")  # Print the full description

# Function to plot MA and MACD indicators
def plot_ma_macd(data, symbol):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # Plot Close Price and Moving Averages
    ax1.plot(data.index, data['Close'], label='Close Price', color='blue')
    ax1.plot(data.index, data['Short_MA'], label='Short MA', color='red')
    ax1.plot(data.index, data['Long_MA'], label='Long MA', color='green')
    ax1.set_title(f'{symbol} - Close Price and Moving Averages')
    ax1.set_ylabel('Close Price')
    ax1.legend(loc='upper left')
    ax1.grid(True)

    # Plot MACD
    ax2.plot(data.index, data['MACD'], label='MACD', color='blue')
    ax2.plot(data.index, data['MACD_Signal'], label='MACD Signal', color='red')
    ax2.bar(data.index, data['MACD_Hist'], label='MACD Hist', color='grey')
    ax2.set_title('MACD')
    ax2.set_ylabel('MACD Value')
    ax2.legend(loc='upper left')
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

print("The following stocks have met the specified criteria. A plot is displayed showing the stock price and its relationship to the associated indicators. Additionally, detailed financial and company information is provided for each stock.\n")

# Iterate through all tickers that meet the criteria to plot graphs and print financial data
for stock_data in results:
    symbol = stock_data["Symbol"]
    data = stock_data["Data"]
    plot_ma_macd(data, symbol)
    print_financial_data(symbol, stock_data)